# 🧠 Preprocessing for Classification 🛠️
---

* Classification을 위한 뇌파 데이터를 전처리 한다.

In [ ]:
import mne
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

### 🗂️s01 ~ s52 mat파일을 변수로 저장

In [ ]:
slen = 52
mat_list = []

for i in range(0, slen):
    mat_list.append(scipy.io.loadmat("GIST_MI/s{:02}.mat".format(i+1)))

### 🎒```mat``` file을 ```Raw``` 객체로 변환

In [ ]:
# info 매개변수 생성
eeg_ch_names = [
    "Fp1", "AF7", "AF3", "F1", "F3", "F5", "F7", "FT7", "FC5", "FC3", "FC1",
    "C1", "C3", "C5", "T7", "TP7", "CP5", "CP3", "CP1", "P1", "P3", "P5", "P7",
    "P9", "PO7", "PO3", "O1", "Iz", "Oz", "POz", "Pz", "CPz", "Fpz", "Fp2",
    "AF8", "AF4", "AFz", "Fz", "F2", "F4", "F6", "F8", "FT8", "FC6", "FC4",
    "FC2", "FCz", "Cz", "C2", "C4", "C6", "T8", "TP8", "CP6", "CP4", "CP2",
    "P2", "P4", "P6", "P8", "P10", "PO8", "PO4", "O2",
]
emg_ch_names = ["EMG1", "EMG2", "EMG3", "EMG4"]
ch_names = eeg_ch_names + emg_ch_names + ["Stim"]
ch_types = ["eeg"] * 64 + ["emg"] * 4 + ["stim"]
srate = mat_list[0]['eeg'][0]['srate'][0][0]

# info 생성
info = mne.create_info(ch_names=ch_names, sfreq=srate, ch_types=ch_types)

# Stim channel 붙이기
left = []
right = []
for i in range(0, slen):
    left.append(np.vstack((mat_list[i]['eeg'][0]['imagery_left'][0], mat_list[i]['eeg'][0]['imagery_event'][0])))
    right.append(np.vstack((mat_list[i]['eeg'][0]['imagery_right'][0], mat_list[i]['eeg'][0]['imagery_event'][0])))


# Raw 생성
lr = []
rr = []
for i in range(0, slen):
    lr.append(mne.io.RawArray(left[i], info=info))
    rr.append(mne.io.RawArray(right[i], info=info))
raw = [lr, rr]

In [ ]:
LEFT = 0
RIGHT = 1

### 🔖Reference

In [ ]:
for i in range(0, slen):
    mne.set_eeg_reference(raw[LEFT][i], ch_type='eeg')
    mne.set_eeg_reference(raw[RIGHT][i], ch_type='eeg')

### 🚪Bandpass Filtering(5Hz ~ 40Hz)

In [ ]:
for i in range(0, slen):
    raw[LEFT][i].filter(l_freq=5, h_freq=40, n_jobs=-1)
    raw[RIGHT][i].filter(l_freq=5, h_freq=40, n_jobs=-1)

### ✂️-500ms - 3000ms Epoching 

In [ ]:
# events 생성
le = []
re = []
for i in range(0, slen):
    le.append(mne.find_events(raw[LEFT][i], stim_channel='Stim'))
    re.append(mne.find_events(raw[RIGHT][i], stim_channel='Stim'))
events = [le, re]

In [ ]:
# Epochs 생성
le = []
re = []
for i in range(0, slen):
    le.append(mne.Epochs(raw[LEFT][i], events[LEFT][i], tmin=-0.5, tmax=3, baseline=(-0.5, 0)))
    re.append(mne.Epochs(raw[RIGHT][i], events[RIGHT][i], tmin=-0.5, tmax=3, baseline=(-0.5, 0)))
epochs = [le, re]

### ❓Question
* 500ms ~ 3000ms 인지, -500ms ~ 3000ms 인지 ...
    * 500ms ~ 3000ms 라면 baseline correction은 어딜 기준으로 해야 하는가?

### ⭕Classification❌

To be continue...